In [ ]:
from regions import Regions

w51e_b6_adam_unselected = Regions.read('tables/w51e_b6_adam_insignificant.reg', format='ds9')
nazar_unselected = Regions.read('tables/w51e_b6_nazar_insignificant.reg', format='ds9')
taehwa_unselected = Regions.read('tables/w51e_b6_taehwa_insignificant.reg', format='ds9')


def get_insignificant_regions(region_a_file, region_b_file, region_c_file, match_radius=1.1e-5):
    regions_A = Regions.read(region_a_file)
    regions_B = Regions.read(region_b_file)
    regions_C = Regions.read(region_c_file)

    all_regions = {'A': regions_A, 'B': regions_B, 'C': regions_C}

    # Define a match radius for overlap (e.g., 1 arcsec)

    # Function to get center coordinate
    def get_center(region):
        return region.center

    # Store matches: key = region, value = set of other files it overlaps with
    matches = defaultdict(set)

    # Compare across files
    file_names = list(all_regions.keys())

    for i, name1 in enumerate(file_names):
        for j, name2 in enumerate(file_names):
            if i >= j:
                continue  # Avoid self-comparison and double-comparing
            regs1 = all_regions[name1]
            regs2 = all_regions[name2]

            for r1 in regs1:
                c1 = get_center(r1)
                for r2 in regs2:
                    c2 = get_center(r2)
                    sep = c1.separation(c2)
                    if sep < match_radius:
                        matches[r1].add(name2)
                        matches[r2].add(name1)

    # Keep only regions that match with >1 other files
    final_regions = [r for r, matched_files in matches.items() if len(matched_files) >= 2]


    return final_regions


def match_between_insignificant_regions(regions_b3, regions_b6, threshold=1.1e-5):
    """
    Match regions between B3 and B6 based on a threshold distance.
    
    Parameters:
    - regions_b3: Regions object for B3
    - regions_b6: Regions object for B6
    - threshold: Distance threshold for matching (default is 1.1e-5 degrees)
    
    Returns:
    - matched_regions: List of matched regions
    """
    
    matched_regions = []
    
    for region_b3 in regions_b3:
        center_b3 = region_b3.center
        
        for region_b6 in regions_b6:
            center_b6 = region_b6.center
            
            if center_b3.separation(center_b6) < threshold:
                matched_regions.append((region_b3, region_b6))
    
    return matched_regions